# Integrating your Devices with MADSci Nodes

This notebook aims to teach you how to automate and integrate all the devices, instruments, sensors, and robots in your self-driving lab using the MADSci Node standard.

## Goals

After completing this notebook, you should understand

1. What we mean when we talk about a MADSci Node
2. The MADSci Node interface standard
3. How to integrate and automate a device using the MADSci Node standard
4. How to use the `RestNode` python class included in `madsci.node_module` to integrate a MADSci Node


Outline:

- overview of madsci/nodes/terms
- dashboard x workcell demo
- code deep dive

In [ ]:
from fastapi.testclient import TestClient
from madsci.node_module.rest_node_module import RestNode
from rich import print

In [ ]:
class ExampleNode(RestNode):
    """Define an Example Node. It doesn't do anything yet, but it's a good starting point."""

In [ ]:
from madsci.common.types.node_types import NodeDefinition

node_definition = NodeDefinition(
    node_name="example_node",
    module_name="example_node_module",
    description="An example node",
)
example_node = ExampleNode(node_definition=node_definition)
example_node.start_node(testing=True)
test_client = TestClient(example_node.rest_api)
with test_client as request:
    response = request.get("/status")
    print(response.json())

In [ ]:
from collections.abc import Generator
from typing import Any
from unittest.mock import patch

from madsci.client.node.rest_node_client import RestNodeClient


def client(test_client: TestClient) -> Generator[RestNodeClient, None, None]:
    """Fixture for DataClient patched to use TestClient"""
    with patch("madsci.client.node.rest_node_client.requests") as mock_requests:

        def post_no_timeout(*args: Any, **kwargs: Any) -> Any:
            kwargs.pop("timeout", None)
            return test_client.post(*args, **kwargs)

        mock_requests.post.side_effect = post_no_timeout

        def get_no_timeout(*args: Any, **kwargs: Any) -> Any:
            kwargs.pop("timeout", None)
            return test_client.get(*args, **kwargs)

        mock_requests.get.side_effect = get_no_timeout

        yield RestNodeClient(url="http://localhost:2000")

In [ ]:
with (
    test_client as request,
    patch("madsci.client.node.rest_node_client.requests") as mock_requests,
):

    def post_no_timeout(*args: Any, **kwargs: Any) -> Any:
        """Patch post method to remove timeout argument"""
        kwargs.pop("timeout", None)
        return request.post(*args, **kwargs)

    mock_requests.post.side_effect = post_no_timeout

    def get_no_timeout(*args: Any, **kwargs: Any) -> Any:
        """Patch get method to remove timeout argument"""
        kwargs.pop("timeout", None)
        return request.get(*args, **kwargs)

    mock_requests.get.side_effect = get_no_timeout

    rest_node_client = RestNodeClient(url="http://localhost:2000")
    print(rest_node_client.get_status())